In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/02/23 06:43:02 WARN Utils: Your hostname, yujaeseong-ui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 192.168.219.147 instead (on interface en0)
23/02/23 06:43:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/23 06:43:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read \
        .option("header", "true") \
        .csv("../fhv_tripdata_2021-01.csv.gz")

In [4]:
df.head(5)

[Row(dispatching_base_num='B00009', pickup_datetime='2021-01-01 00:27:00', dropoff_datetime='2021-01-01 00:44:00', PULocationID=None, DOLocationID=None, SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00009', pickup_datetime='2021-01-01 00:50:00', dropoff_datetime='2021-01-01 01:07:00', PULocationID=None, DOLocationID=None, SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00013', pickup_datetime='2021-01-01 00:01:00', dropoff_datetime='2021-01-01 01:51:00', PULocationID=None, DOLocationID=None, SR_Flag=None, Affiliated_base_number='B00013'),
 Row(dispatching_base_num='B00037', pickup_datetime='2021-01-01 00:13:09', dropoff_datetime='2021-01-01 00:21:26', PULocationID=None, DOLocationID='72', SR_Flag=None, Affiliated_base_number='B00037'),
 Row(dispatching_base_num='B00037', pickup_datetime='2021-01-01 00:38:31', dropoff_datetime='2021-01-01 00:53:44', PULocationID=None, DOLocationID='61', SR_Flag=None, Affiliated_base_number='B0003

In [5]:
df.schema

StructType(List(StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true),StructField(Affiliated_base_number,StringType,true)))

In [6]:
from pyspark.sql import types

In [7]:
schema = types.StructType([types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.StringType(), True),
    types.StructField('dropoff_datetime', types.StringType(), True),
    types.StructField('PULocationID', types.LongType(), True),
    types.StructField('DOLocationID', types.LongType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)])

In [8]:
df = spark.read \
        .option("header", "true") \
        .schema(schema) \
        .csv("../fhv_tripdata_2021-01.csv.gz")

In [9]:
df.head(10)

[Row(dispatching_base_num='B00009', pickup_datetime='2021-01-01 00:27:00', dropoff_datetime='2021-01-01 00:44:00', PULocationID=None, DOLocationID=None, SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00009', pickup_datetime='2021-01-01 00:50:00', dropoff_datetime='2021-01-01 01:07:00', PULocationID=None, DOLocationID=None, SR_Flag=None, Affiliated_base_number='B00009'),
 Row(dispatching_base_num='B00013', pickup_datetime='2021-01-01 00:01:00', dropoff_datetime='2021-01-01 01:51:00', PULocationID=None, DOLocationID=None, SR_Flag=None, Affiliated_base_number='B00013'),
 Row(dispatching_base_num='B00037', pickup_datetime='2021-01-01 00:13:09', dropoff_datetime='2021-01-01 00:21:26', PULocationID=None, DOLocationID=72, SR_Flag=None, Affiliated_base_number='B00037'),
 Row(dispatching_base_num='B00037', pickup_datetime='2021-01-01 00:38:31', dropoff_datetime='2021-01-01 00:53:44', PULocationID=None, DOLocationID=61, SR_Flag=None, Affiliated_base_number='B00037'),

In [10]:
df = df.repartition(24)

In [12]:
# df.write.parquet('../fhvhv/2021/01')

In [14]:
df = spark.read.parquet('../fhvhv/2021/01')
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [16]:
df.show(1)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B01312|2021-01-28 16:40:55|2021-01-28 17:28:30|        null|         121|   null|                B03136|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
only showing top 1 row



In [17]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .filter(df.dispatching_base_num == 'B01312')

DataFrame[pickup_datetime: string, dropoff_datetime: string, PULocationID: bigint, DOLocationID: bigint]

In [18]:
from pyspark.sql import functions as F

In [21]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show(5)

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-28|  2021-01-28|        null|         121|
| 2021-01-29|  2021-01-29|        null|         228|
| 2021-01-16|  2021-01-16|        null|         225|
| 2021-01-06|  2021-01-06|         123|        null|
| 2021-01-16|  2021-01-16|        null|         197|
+-----------+------------+------------+------------+
only showing top 5 rows



In [23]:
def crazy_stuff(base_num):
    # base_num = 'B01312'
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [24]:
crazy_stuff('B02884')

's/b44'

In [28]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [30]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show(5)

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/520| 2021-01-28|  2021-01-28|        null|         121|
|  s/afe| 2021-01-29|  2021-01-29|        null|         228|
|  s/b21| 2021-01-16|  2021-01-16|        null|         225|
|  e/1c1| 2021-01-06|  2021-01-06|         123|        null|
|  a/384| 2021-01-16|  2021-01-16|        null|         197|
+-------+-----------+------------+------------+------------+
only showing top 5 rows

